# Cómo crear embeddings con langchain

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

emb = embeddings.embed_query("hi")
emb

In [ ]:
len(emb)

# Qdrant

In [2]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [3]:
import os

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

In [5]:
client = QdrantClient(
    url=url,
    api_key=api_key
)
client.create_collection(
    collection_name="make-collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

True

In [ ]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="dev-collection-test",
    embedding=embeddings,
)

# Ingest Documents

In [ ]:
import uuid
from langchain_core.documents import Document

In [ ]:
vector_store.add_documents(
    documents=[Document(page_content="I love programming!", metadata={"theme": "software"})], 
    ids=[str(uuid.uuid4())])

# Make a search

In [ ]:
vector_store.similarity_search(query="programming", k=2)

# Use filters

In [ ]:
from qdrant_client import models

results = vector_store.similarity_search(
    query="programming",
    k=1,
    filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.theme",
                match=models.MatchValue(
                    value="software"
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")